## Necessary installations


In [1]:
!pip install -qU transformers einops llama-index llama-index-postprocessor-jinaai-rerank  llama-index-llms-huggingface "huggingface_hub[inference]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-huggingface-api

In [21]:
##--------------------------Libaries----------------------------------------------------------------------------------
import getpass

from typing import List, Tuple

from transformers import AutoModelForSequenceClassification
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core import PromptTemplate

In [5]:
# Download the videogames DB
!wget https://github.com/bbrumm/databasestar/raw/main/sample_databases/sample_db_videogames/sqlite/videogames.db

--2024-11-01 15:34:57--  https://github.com/bbrumm/databasestar/raw/main/sample_databases/sample_db_videogames/sqlite/videogames.db
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bbrumm/databasestar/main/sample_databases/sample_db_videogames/sqlite/videogames.db [following]
--2024-11-01 15:34:57--  https://raw.githubusercontent.com/bbrumm/databasestar/main/sample_databases/sample_db_videogames/sqlite/videogames.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1974272 (1.9M) [application/octet-stream]
Saving to: ‘videogames.db’

videogames.db       100%[===================>]   1.88M  --.-KB/s 

In [6]:
# Connect to the HuggingFace API
print("Paste your Hugging Face access token here: ")
hf_token = getpass.getpass()

Paste your Hugging Face access token here: 
··········


In [7]:
# Load the Generator Model

mistral_llm = HuggingFaceInferenceAPI(model_name="mistralai/Mixtral-8x7B-Instruct-v0.1", token=hf_token)


answer = mistral_llm.complete("this is the lord of rings movie , It is")

answer.text

" a fantasy film series consisting of three high fantasy adventure films based on English author J. R. R. Tolkien's The Lord of the Rings, a trilogy of epic fantasy novels. The films are subtitled The Fellowship of the Ring (2001), The Two Towers (2002), and The Return of the King (2003). The series was directed by Peter Jackson and written by Fran Walsh, Philippa Boyens, and Jackson. The films are a New Zealand-United States co-production.\n\nThe story is set in the fictional world of Middle-earth, and follows the hobbit Frodo Baggins (Elijah Wood) as he and a diverse group of companions embark on a perilous journey to destroy the One Ring, a powerful and malevolent tool created by the Dark Lord Sauron, who seeks to use it to conquer and enslave all of Middle-earth.\n\nThe Lord of the Rings film series was a major financial success, with the films collectively grossing over $2.9 billion in worldwide box office receipts. It was also a critical success, with the series winning"

In [8]:
# Setting the Renraker " Jina Reranker v2"

Reranker = AutoModelForSequenceClassification.from_pretrained(
    "jinaai/jina-reranker-v2-base-multilingual",
    torch_dtype= "auto",
    trust_remote_code = True,

)
# Model should run on cuda
Reranker.to("cuda")
Reranker.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

configuration_xlm_roberta.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- configuration_xlm_roberta.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_xlm_roberta.py:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- mlp.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- block.py
- mlp.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


xlm_padding.py:   0%|          | 0.00/9.82k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- xlm_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- modeling_xlm_roberta.py
- embedding.py
- mha.py
- block.py
- xlm_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768)
      (token_type_embeddings): Embedding(1, 768)
    )
    (emb_drop): Dropout(p=0.1, inplace=False)
    (emb_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): XLMRobertaEncoder(
      (layers): ModuleList(
        (0-11): 12 x Block(
          (mixer): MHA(
            (Wqkv): LinearResidual(in_features=768, out_features=2304, bias=True)
            (inner_attn): SelfAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (inner_cross_attn): CrossAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (dropout1): Dropout(p=0.1, inplace=False)
          (drop_path1): StochasticDepth(p=0.0, mode=r

In [9]:
# The 8 tables definiton from the dump file
table_declarations = [
    "CREATE TABLE platform (\n\tid INTEGER PRIMARY KEY,\n\tplatform_name TEXT DEFAULT NULL\n);",
    "CREATE TABLE genre (\n\tid INTEGER PRIMARY KEY,\n\tgenre_name TEXT DEFAULT NULL\n);",
    "CREATE TABLE publisher (\n\tid INTEGER PRIMARY KEY,\n\tpublisher_name TEXT DEFAULT NULL\n);",
    "CREATE TABLE region (\n\tid INTEGER PRIMARY KEY,\n\tregion_name TEXT DEFAULT NULL\n);",
    "CREATE TABLE game (\n\tid INTEGER PRIMARY KEY,\n\tgenre_id INTEGER,\n\tgame_name TEXT DEFAULT NULL,\n\tCONSTRAINT fk_gm_gen FOREIGN KEY (genre_id) REFERENCES genre(id)\n);",
    "CREATE TABLE game_publisher (\n\tid INTEGER PRIMARY KEY,\n\tgame_id INTEGER DEFAULT NULL,\n\tpublisher_id INTEGER DEFAULT NULL,\n\tCONSTRAINT fk_gpu_gam FOREIGN KEY (game_id) REFERENCES game(id),\n\tCONSTRAINT fk_gpu_pub FOREIGN KEY (publisher_id) REFERENCES publisher(id)\n);",
    "CREATE TABLE game_platform (\n\tid INTEGER PRIMARY KEY,\n\tgame_publisher_id INTEGER DEFAULT NULL,\n\tplatform_id INTEGER DEFAULT NULL,\n\trelease_year INTEGER DEFAULT NULL,\n\tCONSTRAINT fk_gpl_gp FOREIGN KEY (game_publisher_id) REFERENCES game_publisher(id),\n\tCONSTRAINT fk_gpl_pla FOREIGN KEY (platform_id) REFERENCES platform(id)\n);",
    "CREATE TABLE region_sales (\n\tregion_id INTEGER DEFAULT NULL,\n\tgame_platform_id INTEGER DEFAULT NULL,\n\tnum_sales REAL,\n   CONSTRAINT fk_rs_gp FOREIGN KEY (game_platform_id) REFERENCES game_platform(id),\n\tCONSTRAINT fk_rs_reg FOREIGN KEY (region_id) REFERENCES region(id)\n);",
]

In [16]:
# reranker function

def rank_tables(query : str, table_specs: List[str], reranker_model, top_n: int =0) ->List[Tuple[float, str]]:
   """
   Get sorted pairs of scores & table speciifications, then return the top N,
   or all if top_n is 0 or default.
   """

   pairs = [[query, table_spec] for table_spec in table_specs]
   scores = reranker_model.compute_score(pairs)
   scored_tables = [(score, table_spec) for score, table_spec in zip(scores, table_specs )]
   scored_tables.sort(key=lambda x: x[0], reverse=True)

   if top_n and top_n < len(scored_tables):
     return scored_tables[0:top_n]

   return scored_tables




### Test : Reranker

In [14]:
user_query = "Identify the top 10 platforms by total sales."

In [17]:
ranked_tables = rank_tables(user_query, table_declarations, Reranker, top_n=3)

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [18]:
ranked_tables

[(0.5679365992546082,
  'CREATE TABLE region_sales (\n\tregion_id INTEGER DEFAULT NULL,\n\tgame_platform_id INTEGER DEFAULT NULL,\n\tnum_sales REAL,\n   CONSTRAINT fk_rs_gp FOREIGN KEY (game_platform_id) REFERENCES game_platform(id),\n\tCONSTRAINT fk_rs_reg FOREIGN KEY (region_id) REFERENCES region(id)\n);'),
 (0.429668664932251,
  'CREATE TABLE platform (\n\tid INTEGER PRIMARY KEY,\n\tplatform_name TEXT DEFAULT NULL\n);'),
 (0.2689414322376251,
  'CREATE TABLE game_platform (\n\tid INTEGER PRIMARY KEY,\n\tgame_publisher_id INTEGER DEFAULT NULL,\n\tplatform_id INTEGER DEFAULT NULL,\n\trelease_year INTEGER DEFAULT NULL,\n\tCONSTRAINT fk_gpl_gp FOREIGN KEY (game_publisher_id) REFERENCES game_publisher(id),\n\tCONSTRAINT fk_gpl_pla FOREIGN KEY (platform_id) REFERENCES platform(id)\n);')]

In [23]:
## PromptTemplate : to generate a sql query based on retrived tables

sql_prompt_template_text = """
Generate a SQL query to answer the following question from the user :
\"{query_str}\"

The SQL query should use only tables with the following SQL definitions:

Table 1:
{table_1}

Table 2:
{table_2}

Table 3:
{table_3}

You must output only the SQL query with no additional text.

"""

sql_template_prompt = PromptTemplate(sql_prompt_template_text)


In [29]:
ranked_tables[2][1]

'CREATE TABLE game_platform (\n\tid INTEGER PRIMARY KEY,\n\tgame_publisher_id INTEGER DEFAULT NULL,\n\tplatform_id INTEGER DEFAULT NULL,\n\trelease_year INTEGER DEFAULT NULL,\n\tCONSTRAINT fk_gpl_gp FOREIGN KEY (game_publisher_id) REFERENCES game_publisher(id),\n\tCONSTRAINT fk_gpl_pla FOREIGN KEY (platform_id) REFERENCES platform(id)\n);'

In [31]:
## fill in the template

sql_prompt = sql_template_prompt.format(
    query_str=user_query, table_1=ranked_tables[0][1],table_2=ranked_tables[1][1], table_3=ranked_tables[2][1]
)

print(sql_prompt)


Generate a SQL query to answer the following question from the user :
"Identify the top 10 platforms by total sales."

The SQL query should use only tables with the following SQL definitions:

Table 1:
CREATE TABLE region_sales (
	region_id INTEGER DEFAULT NULL,
	game_platform_id INTEGER DEFAULT NULL,
	num_sales REAL,
   CONSTRAINT fk_rs_gp FOREIGN KEY (game_platform_id) REFERENCES game_platform(id),
	CONSTRAINT fk_rs_reg FOREIGN KEY (region_id) REFERENCES region(id)
);

Table 2:
CREATE TABLE platform (
	id INTEGER PRIMARY KEY,
	platform_name TEXT DEFAULT NULL
);

Table 3:
CREATE TABLE game_platform (
	id INTEGER PRIMARY KEY,
	game_publisher_id INTEGER DEFAULT NULL,
	platform_id INTEGER DEFAULT NULL,
	release_year INTEGER DEFAULT NULL,
	CONSTRAINT fk_gpl_gp FOREIGN KEY (game_publisher_id) REFERENCES game_publisher(id),
	CONSTRAINT fk_gpl_pla FOREIGN KEY (platform_id) REFERENCES platform(id)
);

You must output only the SQL query with no additional text.




In [32]:
response = mistral_llm.complete(sql_prompt)
sql_query = str(response)

print(sql_query)

SELECT platform.platform_name, SUM(region_sales.num_sales) AS total_sales
FROM region_sales
JOIN game_platform ON region_sales.game_platform_id = game_platform.id
JOIN platform ON game_platform.platform_id = platform.id
GROUP BY platform.platform_name
ORDER BY total_sales DESC
LIMIT 10;


In [35]:
## Running the SQL query

import sqlite3

con = sqlite3.connect("videogames.db")

cur = con.cursor()

sql_response = cur.execute(sql_query).fetchall()

print(sql_response)

[('PS2', 1233.5599999999467), ('X360', 972.1799999999757), ('PS3', 949.3799999999832), ('Wii', 909.1999999999789), ('DS', 817.939999999971), ('PS', 727.5599999999823), ('GBA', 312.8799999999974), ('PSP', 291.65999999999724), ('PS4', 278.15999999999946), ('PC', 254.6199999999989)]


In [36]:
## Getting a Natural Language Answer

rag_promopt_tmplt_str = """

Use the information in the JSON table to answer the following user query.
Do not explain anything, just answer concisely. Use natural language in
answer, not computer formatting.

USER QUERY: {query_str}

JSON table:
{json_table}

This table was generated by the following SQL query:
{sql_query}

Answer ONLY using the information in the table and the SQL query, and if the
table does not provide the information to answer the question, answer
"No Information".
"""
rag_prompt_tmpl = PromptTemplate(rag_promopt_tmplt_str)


In [37]:
import json

rag_prompt = rag_prompt_tmpl.format(
    query_str="Identify the top 10 platforms by total sales",
    json_table=json.dumps(sql_response), sql_query=sql_query
)

In [38]:
rag_reponse = mistral_llm.complete(rag_prompt)

print(str(rag_reponse))


Answer:

PS2, X360, PS3, Wii, DS, PS, GBA, PSP, PS4, PC


#### Answer user QUery function

In [41]:
def answer_sql(user_query: str) -> str:

  try:
    ranked_tables = ranked_tables = rank_tables(user_query, table_declarations, Reranker, top_n=3)

  except Exception as e:
    print(f"Ranking failed. \nUser query: \n{user_query}\n\n")
    raise (e)

  # Make the Sql template

  make_sql_prompt = sql_template_prompt.format(
      Query_str=user_query, table_1=ranked_tables[0][1],table_2=ranked_tables[1][1], table_3=ranked_tables[2][1]
  )
  try:
    response = mistral_llm.complete(make_sql_prompt)
  except Exception as e:
    print(f"SQL query generation failed\nPrompt:\n{make_sql_prompt}\n\n")
    raise (e)


    ## Backslash removal :Sometimes Mistral puts them in its generated code.
  sql_query = str(response).replace("\\", "")

  try:

    sql_response = sqlite3.connect("videogames.db").cursor().execute(sql_query).fetchall()
  except Exception as e:
    print(f"SQL querying failed. Query:\n{sql_query}")
    raise (e)

  rag_prompt =  rag_prompt_tmpl.format(
      query_str=user_query, json_table=json.dumps(sql_response), sql_query=sql_query
  )
  try:
    rag_response = mistral_llm.complete(rag_prompt)
  except Exception as e:
    print(f"RAG prompt generation failed\nPrompt:\n{rag_prompt}\n\n")
    raise (e)

  return str(rag_response)



In [42]:
print(answer_sql("List the publisher with the largest number of published games."))


Answer:

The publisher with the largest number of published games is Ubisoft with 111 games.


In [43]:
print(answer_sql("What is the most popular game genre of 2012?"))


Answer:

The most popular game genre of 2012 is "Action".
